In [1]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

In [2]:
from psql_helpers import read_table_async

In [3]:
from psql_models import User,Content

In [4]:
from psql_access import get_meeting_token
from datetime import datetime
from indexing.redis_keys import RedisKeys
from indexing.worker import IndexingWorker
from redis import Redis


Connecting to Elasticsearch at elasticsearch:9200


In [5]:
from sqlalchemy import update
from psql_helpers import get_session

async def reset_content_indexing():
    async with get_session() as session:
        await session.execute(
            update(Content).values(is_indexed=False)
        )
        await session.commit()

# Execute
await reset_content_indexing()

In [6]:
df = await read_table_async(Content)

In [7]:
df

,id,content_id,type,text,timestamp,parent_id,is_indexed,last_update
0,1d2eb058-1575-4cb4-b892-a5be7f4d6b4e,1d2eb058-1575-4cb4-b892-a5be7f4d6b4e,meeting,None,2025-01-24 13:25:21,None,False,2025-01-24 13:55:21
1,1abb7019-969c-485a-8a01-45e1225d911c,1abb7019-969c-485a-8a01-45e1225d911c,meeting,None,2025-01-24 07:33:22,None,False,2025-01-24 07:54:43
2,ee6c1dc7-b1e7-4f28-aa47-806dc1201ec0,ee6c1dc7-b1e7-4f28-aa47-806dc1201ec0,meeting,None,2025-01-24 09:22:41,None,False,2025-01-24 09:25:46
3,39dbbf4a-686a-491f-bb6d-da5946ceefc2,39dbbf4a-686a-491f-bb6d-da5946ceefc2,meeting,None,2025-01-24 07:40:43,None,False,2025-01-24 08:45:26
4,24cd7b6f-66cc-4c2b-b095-4aeba089ade3,24cd7b6f-66cc-4c2b-b095-4aeba089ade3,meeting,None,2025-01-24 08:34:58,None,False,2025-01-24 09:05:02
5,7268dd8c-bc04-4af0-8013-5b7e310595fe,7268dd8c-bc04-4af0-8013-5b7e310595fe,meeting,None,2025-01-24 08:00:03,None,False,2025-01-24 09:16:20
6,cbe2de7e-7c41-40e1-bbd9-b42c771037e2,cbe2de7e-7c41-40e1-bbd9-b42c771037e2,meeting,None,2025-01-23 11:04:52,None,False,2025-01-24 11:32:55
7,8ecdb9f0-03ad-44f5-9d83-d7c55664fbf3,8ecdb9f0-03ad-44f5-9d83-d7c55664fbf3,meeting,None,2025-01-24 08:01:27,None,False,2025-01-24 11:54:08
8,e94f95e8-c226-43b0-9c23-65f82b42abd1,e94f95e8-c226-43b0-9c23-65f82b42abd1,meeting,None,2025-01-24 10:59:59,None,False,2025-01-24 11:08:50
9,13ad56a5-c97f-49f3-9da4-b6be4f9f59e8,13ad56a5-c97f-49f3-9da4-b6be4f9f59e8,meeting,None,2025-01-24 10:03:03,None,False,2025-01-24 10:33:32


In [8]:
df['is_indexed'].value_counts()


is_indexed
False    3692
Name: count, dtype: int64

In [29]:
df['type'].value_counts()


type
meeting    3687
title         4
Name: count, dtype: int64

In [30]:
processor = ContentProcessor(
            qdrant_engine=self.qdrant_engine,
            es_engine=self.es_engine
        )

NameError: name 'self' is not defined